# High-dimensional Bayesian Optimization of CrabNet Hyperparameters
###### Created March 5, 2022

![logo](matbench_logo.png)


# Description

Use of hyperparameter Bayesian optimization with [CrabNet](https://crabnet.readthedocs.io/) revealed favorable results in
improving the error over the baseline model; however, only 100 iterations were used to
optimize 23 hyperparameters with little prior information to make an informed decision
(for example, the default CrabNet hyperparameters were not available to the
hyperparameter optimization algorithm). Only the constraints on the hyperparameter
search space, which were quite generous, would constitute prior information, and 100
iterations across 23 parameters is a very sparse sampling. Here, we describe a more direct comparison to
the relatively new high-dimensional Bayesian optimization algorithm called Sparse
Axis-Aligned Subspace Bayesian Optimization, or SAASBO. We use 10 Sobol trials and 90 Bayesian trials with the default Gaussian Process Expected Improvement (GPEI) model to be more comparable to the results with SAASBO which also used 10 Sobol trials and 90 Bayesian trials.

[facebook/Ax](https://github.com/facebook/Ax) is used as the backend for performing
SAASBO. For additional files related to this `matbench` submission, see the
[crabnet-hyperparameter](https://github.com/sparks-baird/crabnet-hyperparameter)
repository. Due to the procedure being relatively expensive (esp. due to use of
nested-CV when training CrabNet) and requiring a few days of runtime on two RTX 2080 Ti
GPUs, this notebook demonstrates how to submit jobs to a high-performance computing
center (in our case, CHPC @ University of Utah). `submitit` parameters such as `account`
and `partition` might vary somewhat for your own HPC center.

# Benchmark name
Matbench v0.1

# Package versions
- matplotlib==3.5.0
- pandas==1.3.5
- ax-platform==0.2.3
- pyro-ppl==1.8.0
- plotly==5.5.0
- crabnet==1.2.5
- scikit-learn
- submitit==1.4.1
- matbench==0.5
- cloudpickle==2.0.0

# Algorithm description
Recently, SAASBO has been demonstrated to be a highly effect high-dimensional Bayesian
optimization scheme. Here, we compare against [Ax/SAASBO Bayesian adaptive design](https://ax.dev/tutorials/saasbo.html) to simultaneously optimize 23
hyperparameters of
[CrabNet](https://crabnet.readthedocs.io/). `100` sequential design iterations were used, and parameters were chosen based
on a combination of intuition and algorithm/data constraints (e.g. elemental featurizers
which were missing elements contained in the dataset were removed). The first `10`
iterations were based on SOBOL sampling to create a rough initial model, while the
remaining `90` iterations were GPEI Bayesian adaptive design iterations. For the inner
loops (where hyperparameter optimization is performed), the average MAE across each of
the *five inner folds* was used as Ax's objective to minimize. The best parameter set
was then trained on all the inner fold data and used to predict on the test set (unknown
during hyperparameter optimization). This is nested cross-validation (CV), and is
computationally expensive. See [automatminer: running a
benchmark](https://hackingmaterials.lbl.gov/automatminer/advanced.html#running-a-benchmark)
for more information on nested CV.

# Relevant citations
- CrabNet: [Wang et al.](https://doi.org/10.1038/s41524-021-00545-1)
- SAASBO: [Eriksson and Jankowiak](
https://doi.org/10.48550/arXiv.2103.00349)

# Related Matbench Submissions
- [`matbench_v0.1_Ax_SAASBO_CrabNet_v1.2.7`](https://github.com/materialsproject/matbench/tree/main/benchmarks/matbench_v0.1_Ax_CrabNet_v1.2.1)
- [`matbench_v0.1_Ax_CrabNet_v1.2.1`](https://github.com/materialsproject/matbench/tree/main/benchmarks/matbench_v0.1_Ax_CrabNet_v1.2.1)
- [`matbench_v0.1_CrabNet_v1.2.1`](https://github.com/materialsproject/matbench/tree/main/benchmarks/matbench_v0.1_CrabNet_v1.2.1)

# Imports

In [1]:
# %% imports
# NOTE: `pip install pyro-ppl` to use FULLYBAYESIAN (SAASBO)
from submitit import AutoExecutor
import cloudpickle as pickle
from utils.matbench import matbench_fold_GPEI, collect_results, task, savepath, dummy
print(f"dummy: {dummy}")

# Job Submission

In [ ]:
# %% submission
log_folder = "log_ax/%j"
walltime = 28 * 60  # 4320  # 4320 min == 3 days
# partition, account = ["notchpeak-gpu-guest", "owner-gpu-guest"]
partition, account = ["notchpeak-gpu", "notchpeak-gpu"]
# partition, account = ["notchpeak-guest", "owner-guest"]
executor = AutoExecutor(folder=log_folder)
executor.update_parameters(
    timeout_min=walltime,
    slurm_partition=partition,
    slurm_gpus_per_task=1,
    slurm_mem_per_gpu=6000,
    slurm_cpus_per_gpu=4,
    slurm_additional_parameters={"account": account},
)
jobs = executor.map_array(matbench_fold_GPEI, task.folds)  # sbatch array
job_ids = [job.job_id for job in jobs]
# https://www.hpc2n.umu.se/documentation/batchsystem/job-dependencies
job_ids_str = ":".join(job_ids)  # e.g. "3937257_0:3937257_1:..."

with open("jobs.pkl", "wb") as f:
    pickle.dump(jobs, f)

# Collect Results

In [ ]:
collect_folder = "log_matbench/%j"
walltime = 10
collector = AutoExecutor(folder=collect_folder)
collector.update_parameters(
    timeout_min=walltime,
    slurm_partition=partition,
    slurm_additional_parameters={
        "account": account,
        "dependency": f"afterok:{job_ids_str}",
    },
)
collector_job = collector.submit(collect_results, False)  # sbatch array

print(
    f"Waiting for submission jobs ({job_ids_str}) to complete before running collector job ({collector_job.job_id}). Use the matbench output file that will be saved to {savepath} after all jobs have run."
)